In [ ]:
# *** PROBLEM 1 *** # 
# a
success_probability = 4/52 
print("a)", success_probability)

# b
def geometric_distribution(p, k): 
    return ((1-p)**(k-1)) * (p)

print("b)", geometric_distribution(success_probability, 4)) 

# c 
import numpy as np

results = np.random.geometric(success_probability, 100000)

print("c)", results)

# d
counter = 0
for trial in results: 
    if (trial == 4):
        counter += 1

print("d)", counter/100000) 

# e 
experimental_results = [15, 6, 13, 14, 1, 10, 5, 2, 11, 3, 
                        32, 8, 4, 4, 7, 20, 13, 12, 8, 4, 
                        14, 15, 4, 37, 3, 2, 6, 12, 11, 5]

print("e)", experimental_results)

# f
counter = 0
for trial in experimental_results: 
    if (trial == 4): 
        counter += 1 

print("f)", counter/30)

#a) 0.07692307692307693
#b) 0.060502083260390055
#c) [5 7 3 ... 4 1 5]
#d) 0.06103
#e) [15, 6, 13, 14, 1, 10, 5, 2, 11, 3, 32, 8, 4, 4, 7, 20, 13, 12, 8, 4, 14, 15, 4, 37, 3, 2, 6, 12, 11, 5]
#f) 0.13333333333333333

a) 0.07692307692307693
b) 0.060502083260390055
c) [5 7 3 ... 4 1 5]
d) 0.06103
e) [15, 6, 13, 14, 1, 10, 5, 2, 11, 3, 32, 8, 4, 4, 7, 20, 13, 12, 8, 4, 14, 15, 4, 37, 3, 2, 6, 12, 11, 5]
f) 0.13333333333333333


# g: 

Theoretical Probability: 0.0605 = 6.05% 
Python Sim: 0.0610 = 6.1%
Physical Experiment: 0.13 = 13.3% 

    These results show that the Python simulation and the theoretical probability are very close, 
but the physical experiment resulted in a much higher rate of 4's than expected. This is in part
due to the small sample size, (30 trials vs 100000) and due to the law of large numbers we would 
expect the frequency of 4's to decrease if we increased the number of trials. The physical experiment
also includes a degree of human error when factoring in the shuffling procedure, two different shuffles
could affect the order of the deck in very different ways, so a slight deviation from the expectation 
is not unusual. 